<a href="https://colab.research.google.com/github/NityaVerma19/Movie-recommendation/blob/master/EDA_of_original_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'netflix-prize-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1636%2F792972%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240413%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240413T062152Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D96e4f09d3b82676ae31a9de3e91fadeccb75458ea6390ec36fddcbaca6754f534cb3e07656701f473ebfb1aba0f28245f58de712ad8dbcbeb9c8b882e5fd16480ff952e8327867d7da0ec1c9f6ba357aefdeb6ed6355aaba210a099d92aed502633a60b95ec2117491ce1a215a4ce0b6ab38163ba7be9f12f84d58b9eb50d502767e6943c5bc1aa97c972bb763d4065ffec1c589c098eb7a05ffb45390048eae1e2b60ced4235e2c244f6d589bf0e6ca83813d4df64354151271e3384890fd40997dcf7ac7c0dab84b61cfcc067bae30a97afb3ba897407937a40bc2add79f410aefc5dac7e17757891236a210600277b17dcf99209bc535e04b44f7096c58b6'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 716193814 bytes downloaded
Downloaded and uncompressed: netflix-prize-data
Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/netflix-prize-data/combined_data_4.txt
/kaggle/input/netflix-prize-data/combined_data_1.txt
/kaggle/input/netflix-prize-data/probe.txt
/kaggle/input/netflix-prize-data/combined_data_3.txt
/kaggle/input/netflix-prize-data/combined_data_2.txt
/kaggle/input/netflix-prize-data/README
/kaggle/input/netflix-prize-data/movie_titles.csv
/kaggle/input/netflix-prize-data/qualifying.txt


In [11]:
# Defining column names
column_names = ['MovieID', 'YearOfRelease', 'Title']


# Reading CSV file without headers, skipping lines with parsing errors
movie_titles = pd.read_csv('/kaggle/input/netflix-prize-data/movie_titles.csv', encoding='ISO-8859-1', header=None, names=column_names,on_bad_lines ='skip')


# *Movie Titles*

In [14]:
movie_titles['Title'] = movie_titles['Title'].apply(lambda x:x.lower())

*Dropping Year of release*

In [15]:
movie_titles.drop(columns = ['YearOfRelease'], inplace = True)

In [16]:
movie_titles.head()

,MovieID,Title
0,1,dinosaur planet
1,2,isle of man tt 2004 review
2,3,character
3,4,paula abdul's get up & dance
4,5,the rise and fall of ecw


*Finding number of duplicates*

In [23]:
movie_titles['Title'].duplicated().sum()

411

*Dropping the duplicates*

In [24]:
movie_titles.drop_duplicates(subset = ['Title'], inplace = True)

*Number of movies*

In [25]:
movie_titles.nunique()

MovieID    17023
Title      17023
dtype: int64

Movie_title contains `17023 movies`

# *DATA 1*

In [26]:

def read_combined_data(file_path, chunksize = 10000):
    # Initialize lists to store data
    movie_ids = []
    customer_ids = []
    ratings = []
    dates = []

    # Read the file
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Initialize variables
    current_movie_id = None

    # Process each line
    for line in lines:
        # Strip leading/trailing whitespaces
        line = line.strip()

        # Check for movie ID
        if line.endswith(':'):
            # Update current movie ID
            current_movie_id = line[:-1].strip()
        elif line:  # Check if the line is not empty
            # Extract customer data
            data = line.strip().split(',')
            if len(data) >= 3:
                customer_id = data[0].strip()
                rating_str = data[1].strip()
                date = data[2].strip()

                try:
                    rating = float(rating_str)
                except ValueError:
                    print(f"Skipping invalid rating (not a valid float): '{rating_str}'")
                    continue

                # Append data to lists
                movie_ids.append(current_movie_id)
                customer_ids.append(customer_id)
                ratings.append(rating)
                dates.append(date)
            else:
                print(f"Skipping line with insufficient data: '{line}'")

    # Create DataFrame
    combined_data = pd.DataFrame({
        'MovieID': movie_ids,
        'CustomerID': customer_ids,
        'Rating': ratings,
        'Date': dates
    })

    return combined_data

# Read combined_data_1.txt
file_path = "/kaggle/input/netflix-prize-data/combined_data_1.txt"
print(f"Reading file: {file_path}")
data_1 = read_combined_data(file_path, chunksize = 10000)

# Display the first few rows of the DataFrame
print(data_1.head())

Reading file: /kaggle/input/netflix-prize-data/combined_data_1.txt
  MovieID CustomerID  Rating        Date
0       1    1488844     3.0  2005-09-06
1       1     822109     5.0  2005-05-13
2       1     885013     4.0  2005-10-19
3       1      30878     4.0  2005-12-26
4       1     823519     3.0  2004-05-03


In [ ]:
data_1.head()